In [6]:
import os, re
import sys
import glob
import datetime as dt
from pathlib import Path
from dateutil.parser import parse

import numpy as np
import pandas as pd
import pickle as pkl
import numexpr as ne
import logging

In [7]:
def set_logger(level=logging.INFO, name=__name__):
    logger = logging.getLogger(__class__.__qualname__) or logging.getLogger(name)
    logger.setLevel(level)
    handler = logging.FileHandler('log_file.log')
    formatter = logging.Formatter(
        '%(asctime)s : %(name)s  : %(funcName)s : %(levelname)s : %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger


In [8]:
# PATH = os.path.abspath(r"../data/10X/cleaned/1998/QTR2/") # on home
# PATH = os.path.abspath(r"C:\Users\wonhyeong\workings\data\10X\cleaned") # on office
PATH = os.path.abspath(r"/Users/wonhyeong/workings/data/10X/cleaned")  # on mac
dict_dir = "data/summaries.csv"
summaries = pd.read_csv(dict_dir)[['CIK', 'FILING_DATE', 'ACC_NUM', 'FORM_TYPE', 'CoName']]


In [157]:
sample_dir = r'/Volumes/GoogleDrive/내 드라이브/workspace'

# get directory tree
def get_dir_tree(dir):
    tree = pd.DataFrame(columns=["name", "path", "depth", "files"])
    for root, dirs, files in os.walk(dir):
        name = os.path.basename(root)
        this = root.replace(dir, "")
        depth = int(this.count(os.sep))
        count = len(files)
        # 숨김 폴더거나, 그 자식 노드일 경우 스킵
        if any(f.startswith(".") for f in this.split(os.sep)): 
            print("skip:", this)
            continue
        tree.loc[len(tree)] = [name, this, depth, count]
    return tree


def get_dir_index(dir):
    index = pd.DataFrame(columns=["name", "path", "is_dir"])
    for root, dirs, files in os.walk(dir):
        path = root.replace(dir, "")
        name = os.path.basename(path)
        # 숨김 폴더거나, 그 자식 노드일 경우 스킵
        if any(f.startswith(".") for f in path.split(os.sep)): 
            print("skip:", path)
            continue
        index.loc[len(index), :] = [name, path, True]
        for f in files:
            if f.startswith("."): continue
            name = f
            path_f = os.sep.join([path, f])
            index.loc[len(index), :] = [name, path_f, False]
    return index


def visualize_tree(tree):
    tree = get_dir_tree(sample_dir)
    height = len(tree)
    width = tree.depth.max() + 2
    df = pd.DataFrame(np.full((height, width), "ｌ"))
    for row in tree.itertuples():
        df.loc[row.Index, row.depth] = row.name
        df.loc[row.Index, row.depth + 1] = f"files : {row.count}"
    return df


In [5]:
# get cik.csv
a = pd.read_csv('data/cik.csv')
# prettier
a.drop(a.columns[0], axis=1, inplace=True)
# limit exchange to NYSE or NASDAQ
# due to duplicated cik are common in OTC, we need to limit exchange first
a.drop(a.index[(a['exchange'] != 'NYSE') & (
    a['exchange'] != 'Nasdaq')], inplace=True)
# drop all duplicates
a.drop_duplicates(subset='cik', keep='first', inplace=True)
a.drop_duplicates(subset='name', keep='first', inplace=True)
a.drop_duplicates(subset='ticker', keep='first', inplace=True)
# column cik to index
a.set_index('cik', inplace=True)
ticker = a
# add col 'ticker', 'exchange' to summaries


def findticker(cik):
    try:
        result = ticker.at[cik, 'ticker']
    except:
        return
    return result


def findexchange(cik):
    try:
        result = ticker.at[cik, 'exchange']
    except:
        return
    return result


summaries['TICKER'] = summaries['CIK'].map(findticker)
summaries['EXCHANGE'] = summaries['CIK'].map(findexchange)


In [6]:
def find_path_from_date(filing_date):
    filing_date = str(filing_date)
    year = filing_date[:4]
    # month to quarter
    month = int(filing_date[4:6])
    quarter = (month-1) // 3 + 1
    quarter = 'QTR' + str(quarter)
    path = os.path.join(PATH, year, quarter)
    return path

In [179]:
class what:
  
  def __init__(self, what):
    self.dd = self.what()
    self.ee = None
    self.eee()

  def what(self):
    return "wrqwr"

  def print(self):
    print(self.dd)
    return

  def eee(self):
    self.ee = self.dd
    return

w = what("wrqwr")
w.print()
w.ee

wrqwr


'wrqwr'

In [9]:
class spider:
    # import modules
    import os
    import datetime as dt
    from pathlib import Path
    from dateutil.parser import parse

    import numpy as np
    from pickle import dump  # or using hdf5.fixed
    from numexpr import evaluate as ne_eval

    def __init__(self, data_path):
        self.dir = Path(data_path)
        self.summary = pd.read_pickle(
            self.dir / 'summaries.csv') if (self.dir / 'summaries.csv').exists() else None
        self.tree = pd.read_pickle(
            self.dir / 'tree.csv') if (self.dir / 'tree.csv').exists() else None
        self.index = pd.read_pickle(
            self.dir / 'index.csv') if (self.dir / 'index.csv').exists() else None
        self.columns = {
            "cik": self.summary.CIK.values,
            "date": self.summary.FILING_DATE.values,
            "name": self.summary.CoName.values,
            "acc": self.summary.ACC_NUM.values,
            "form": self.summary.FORM_TYPE.values,
            "symbol": self.summary.TICKER.values,
            "exchange": self.summary.EXCHANGE.values
        }
        self.__set_data()

    def __set_data(self):
        if not self.summary:
            self.summary = pd.read_csv(self.dir / 'data' / 'summaries.csv')
        if not self.index:
            self.__get_dir_index()
            self.__get_dir_tree()
        return

    def search(self, *keywords):
        if not keywords:
            return os.listdir(self.dir)
        if len(keywords) > 1:
            return
        # search by keyword
        key, modifier = self._search_preprocess(str(keywords[0]))
        assert modifier or key, "input your keyword"
        matrix = []
        # 조심해야함
        if modifier:
            matrix.append([self._get_from_modifier(mod, value)
                          for mod, value in modifier.items()])
        if key:
            matrix.append([self._get_from_key(key, value)
                          for key, value in key.items()])
        idx = np.multiply.reduce(matrix)
        # to boolean array
        idx = np.array([True if i else False for i in idx[0]])
        # result = summaries[idx][['FILING_DATE', 'ACC_NUM']]
        # result.set_index('ACC_NUM', inplace=True)
        return self.summary[idx]

    def make_df_from_path(self, paths):
        # make dataframe that has colums of date, type, acc, text
        df = pd.DataFrame(columns=["date", "type", "acc", "text"])
        for path in paths:
            file = os.path.basename(path)
            date = file.split("_")[0]
            type = file.split("_")[1]
            acc = file.split("_")[5].replace(".txt", "")
            text = open(path, "r").read()
            # append to df
            df = df.append(pd.DataFrame([[date, type, acc, text]], columns=[
                           "date", "type", "acc", "text"]))
        return df
        # input: list of path
        # get filename from path
        # output:

    def show_tree(self):
        result = ''
        for row in self.tree.itertuples():
            result.append('\t'*row.depth + row.name +
                          f"\tfiles: {row.count}\n")
        return result

    def _search_preprocess(self, keyword):
        # params : string for search
        # return : list of keywords + tags
        key_tags = ['cik', 'date', 'name', 'symbol', 'acc', 'form']
        modifier_tags = ['after', 'before', 'year', 'qtr']
        keyword_list = keyword.split(' ')
        # unknown 인식 및 처리 과정 필요
        unknown = list(filter(lambda x: ':' not in x, keyword_list))
        tag = list(filter(lambda x: ':' in x, keyword_list))
        tag = dict(tuple(x.split(':')) for x in tag) if tag else {}
        key = {k: v for k, v in tag.items() if k in key_tags}
        modifier = {k: v for k, v in tag.items() if k in modifier_tags}
        # 현재 dateparser 적용 안되는 오류
        for k, val in modifier.items():
            val = self._date_parser(val)
        modifier = {k: v for k, v in modifier.items() if v}
        return key, modifier

    def _date_parser(self, date):
        if len(date) == 4:
            if date.startswith(('19', '20')):
                # 연도만 입력된 경우
                return date+'0101'
            if date <= '1231':
                # 월일만 입력된 경우, 분기조건 수정 필요
                year = str(dt.now().year)
                return year + date
        # try 내에서 return 사용 가능?
        try:
            result = parse(date).strftime('%Y%m%d')
        except:
            return None
        return result

    def __get_dir_tree(self):
        tree = pd.DataFrame(columns=["name", "path", "depth", "files"])
        for root, dirs, files in os.walk(self.dir):
            name = os.path.basename(root)
            this = root.replace(self.dir, "")
            depth = int(this.count(os.sep))
            count = len(files)
            # 숨김 폴더거나, 그 자식 노드일 경우 스킵
            if any(f.startswith(".") for f in this.split(os.sep)):
                print("skip:", this)
                continue
            tree.loc[len(tree)] = [name, this, depth, count]
        return tree

    def __get_dir_index(dir):
        index = pd.DataFrame(columns=["name", "path", "is_dir"])
        for root, dirs, files in os.walk(dir):
            path = root.replace(dir, "")
            name = os.path.basename(path)
            # 숨김 폴더거나, 그 자식 노드일 경우 스킵
            if any(f.startswith(".") for f in path.split(os.sep)):
                print("skip:", path)
                continue
            index.loc[len(index), :] = [name, path, True]
            for f in files:
                if f.startswith("."):
                    continue
                name = f
                path_f = os.sep.join([path, f])
                index.loc[len(index), :] = [name, path_f, False]
        return index

    def _get_depth(self, path):
        for root, dir, file in os.walk(path):
            if file and not dir:
                self.depth = root.count(os.sep)-path.count(os.sep)+1
        return "ERROR"

    def _get_subdir(self, path):
        """search for subdir in path
        return list of subdir
        every iteration, check next(os.walk(subdir))[1]
        if empty continue to next iteration"""
        return "ERROR"

    def _get_from_key(self, key, value):
        col = self.columns[key]
        idx = ne_eval(f'(col == {value})')
        return idx

    def _get_from_modifier(self, modifier, value):
        date = self.columns["date"]
        value = str(value)
        after = "10000000"
        before = "99999999"
        if modifier == 'after':
            after = value
        elif modifier == 'before':
            before = value
        elif modifier == 'year':
            after = value[:4] + '0101'
            before = value[:4] + '1231'
        elif modifier == 'qtr':
            after = value
            before = value[:4] + str(int(value[4:6])+3) + value[6:]
        idx = ne_eval(f'({int(after)} < date) & (date < {int(before)})')
        return idx


In [10]:
date = spider(PATH)
a = date.search('cik:861439')
np.count_nonzero(a)

AttributeError: 'NoneType' object has no attribute 'CIK'

In [56]:
# # summaries = summaries.csv['']
# # innocent = ACC_NUM이 중복되지 않은 리스트
# innocent = summaries.drop_duplicates(subset=['ACC_NUM'], keep=False)
# print(len(innocent))
# inn_cik =  innocent['CIK'].value_counts()
# inn_co =  innocent['CoName'].value_counts()
# inn_co
# print(len(inn_cik), len(inn_co))
# # get duplicated acc num
# duplicated = summaries[summaries['ACC_NUM'].duplicated(keep=False)]
# counts = duplicated['ACC_NUM'].value_counts()
# coname = duplicated['CoName'].value_counts()
# cik = duplicated['CIK'].value_counts()
# acc = counts.index
# print(coname)
# print(cik)
# coname
# summaries['CoName']

1094786
42540 57459
UNITED REFINING CO                            680
ENTERGY CORP /DE/                             543
708 GYM LLC                                   543
SOUTHERN CO                                   468
OUTSOURCING SOLUTIONS INC                     461
                                             ... 
Volkswagen Auto Loan Enhanced Trust 2011-1      2
VOLKSWAGEN AUTO LEASE TRUST 2012-A              2
FLEET CREDIT CARD MASTER TRUST II               2
SLC Student Loan Trust 2004-1                   2
BINGO COM INC                                   2
Name: CoName, Length: 1672, dtype: int64
1367306    420
1383094    244
1347185    163
1477336    143
1002761    123
          ... 
1069892      1
1069891      1
1069890      1
1069604      1
1049299      1
Name: CIK, Length: 3009, dtype: int64


0              LOUISIANA LAND & EXPLORATION CO
1          MINNESOTA MINING & MANUFACTURING CO
2              LOUISIANA LAND & EXPLORATION CO
3              LOUISIANA LAND & EXPLORATION CO
4                    BETHLEHEM STEEL CORP /DE/
                          ...                 
1140481                    Umatrin Holding Ltd
1140482           QMIS TBS Capital Group Corp.
1140483                          Linktory Inc.
1140484                           Energem Corp
1140485                            NEOGEN CORP
Name: CoName, Length: 1140486, dtype: object